In [ ]:
# This script is for creating the list of URLs of the videos for transcription

In [4]:
import pandas as pd
from pprint import pprint

In [ ]:
API_KEY = "KEY_HERE"

# Create a YouTube API client
youtube = build('youtube', 'v3', developerKey=API_KEY)

In [5]:
# create dataframe with ['playlist', 'video_url', 'video_title', 'tanscribed']
videos_df = pd.DataFrame(columns=['playlist', 'video_url', 'video_title', 'transcribed'])
videos_df.head()

,playlist,video_url,video_title,transcribed


In [6]:
# create dictionary of playlist urls
# url_base = https://www.youtube.com/playlist?list=
playlist_urls = {'52_weeks' : 'PLBaMLrfToJyybUT18OE3fMomFb9XU0ffC',
                 '52_FAQ' : 'PLBaMLrfToJyxF2RznuIpcLyUpLU3kLueV',
                 '52_second_edition' : 'PLBaMLrfToJyyhebzEduI7AK0nWHMlcDgK',
                 '11_days' : 'PLBaMLrfToJyznPoFK7iyxmsOHqxcRrm5m',
                 'biome' : 'PLBaMLrfToJyyywPKnlV7P--e6VG3umjW6',
                 'interviews' : 'PLBaMLrfToJywZby8_bU4fFsCUAq9BANsz',
                 'MACNA' : 'PLBaMLrfToJyyToilXCR5jLFMoemxwAE8x',
                 'nutrients' : 'PLBaMLrfToJyzCXREqXjsl066OzEgvXfJr'
                 }

playlist_test = {'biome' : 'PLBaMLrfToJyyywPKnlV7P--e6VG3umjW6'}



In [8]:
# Function to retrieve video URLs and titles from a playlist
def get_playlist_videos(playlist_id):
    videos = []
    next_page_token = None

    # Retrieve videos in the playlist
    while True:
        request = youtube.playlistItems().list(
            part="snippet",
            #part="contentDetails",
            playlistId=playlist_id,
            maxResults=100,  # Adjust as needed
            pageToken=next_page_token
        )
        response = request.execute()

        # Extract video URLs from the response
        for item in response['items']:
            video_id = item['snippet']['resourceId']['videoId']
            #video_id = item['contentDetails']['videoId']
            video_url = f"https://www.youtube.com/watch?v={video_id}"
            video_title = item['snippet']['title']
            videos.append({'playlist': playlist_id, 'title': video_title, 'url': video_url, 'video_id':video_id, 'transcirbed': False})

        # Check if there are more pages of results
        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break

    return videos

# Function to retrieve captions for a video
def get_captions(video_id):
    captions = []

    # Retrieve captions for the video
    request = youtube.captions().list(
        part="snippet",
        videoId=video_id
    )
    response = request.execute()

    # Extract captions from the response
    for item in response['items']:
        caption_id = item['id']
        caption_language = item['snippet']['language']
        captions.append({'id': caption_id, 'language': caption_language})

    return captions

# Function to download and parse caption file to extract transcript text
def download_and_parse_caption(caption_id):
    # Download the caption file
    request = youtube.captions().download(
        id=caption_id,
        tfmt="srt"  # Choose the caption format (e.g., "srt", "vtt")
    )
    caption_response = request.execute()

    # Parse the caption file to extract transcript text
    transcript_text = caption_response.decode('utf-8')  # Assuming the response is in UTF-8 encoding

    return transcript_text

In [9]:
### MAIN

for playlist in playlist_test:
    videos = get_playlist_videos(playlist)

print(videos)


NameError: name 'youtube' is not defined

In [ ]:
video_id = "YOUR_VIDEO_ID_HERE"
captions_data = get_captions(video_id)

# Create a directory to save transcripts
if not os.path.exists("transcripts"):
    os.makedirs("transcripts")

# Download and save transcripts for each caption
for caption in captions_data:
    caption_id = caption['id']
    transcript_text = download_and_parse_caption(caption_id)

    # Save transcript text to a file
    file_name = f"transcripts/{video_id}_{caption_id}.txt"
    with open(file_name, "w", encoding="utf-8") as file:
        file.write(transcript_text)

    print(f"Transcript saved to {file_name}")